# DerivKit: DerivativeKit — Tabulated derivatives (advanced)

### Summary
This notebook demonstrates estimating **first derivatives from noisy tabulated data** using `DerivativeKit`
in *tabulated mode*. We use a toy problem with an analytic truth:

- underlying function: $f(x)=\sin(x)$
- truth derivative: $f'(x)=\cos(x)$

We add Gaussian noise to the sampled table and compare two derivative methods across many evaluation points:
- **finite differences + Ridders extrapolation**
- **adaptive local fit**

### Usage
If you prefer to run the same example as a script:

```bash
$ python demo-scripts/09-derivative-kit-tabulated_advanced.py
```

### What it does
- Builds a noisy tabulated dataset (optionally with x-jitter).
- Evaluates $f'(x)$ on an interior grid of $x_0$ values.
- Compares methods to the analytic truth and prints accuracy metrics (RMSE/MAE/MaxAE).
- Produces three plots:
  1. noisy table vs truth
  2. derivative estimates (with internal errors when available) vs truth
  3. method-reported internal error $\widehat{\sigma}_{f'}(x)$ vs $x$

### Notes
- The “internal error” is whatever the method reports when `return_error=True`; it is method-dependent and
  not necessarily a calibrated statistical 1σ uncertainty.

### Requirements
- `derivkit` installed and importable in your Python environment.
- `matplotlib` installed.


In [ ]:
from __future__ import annotations

import sys
from pathlib import Path
from typing import Any

import matplotlib.pyplot as plt
import numpy as np

# Make repo root importable so `utils.style` works when running from demo-scripts/
# If you run this notebook from the derivkit-demos repo root, this should be fine.
sys.path.insert(0, str(Path.cwd().resolve().parents[0]))

from derivkit.derivative_kit import DerivativeKit
from derivkit.derivatives.tabulated_model.one_d import Tabulated1DModel

# Optional styling (if available in your demos repo)
try:
    from utils.style import DEFAULT_COLORS, apply_plot_style
except Exception:
    DEFAULT_COLORS = {"red": None, "yellow": None, "blue": None}
    def apply_plot_style():
        return


## Helper functions

In [ ]:
def sin_func(x: np.ndarray) -> np.ndarray:
    """Retruns a sine function."""
    return np.sin(x)

def df_truth(x: np.ndarray) -> np.ndarray:
    """Returns the analytic derivative of the sine function (a cosine)."""
    return np.cos(x)

def metrics(estimate: np.ndarray, truth: np.ndarray) -> dict[str, float]:
    """Compute RMSE, MAE, and MaxAE between estimate and truth arrays.

    RMSE is the root-mean-square error, computed as sqrt(mean((estimate - truth)^2)).
    MAE is the mean absolute error, computed as mean(|estimate - truth|).
    MaxAE is the maximum absolute error, computed as max(|estimate - truth|).
    """
    m = np.isfinite(estimate) & np.isfinite(truth)
    if not np.any(m):
        return {"rmse": float("nan"), "mae": float("nan"), "maxae": float("nan")}
    err = estimate[m] - truth[m]
    ae = np.abs(err)
    return {
        "rmse": float(np.sqrt(np.mean(err * err))),
        "mae": float(np.mean(ae)),
        "maxae": float(np.max(ae)),
    }

def tex_sci(x: float, sig: int = 2) -> str:
    """Format a float in LaTeX scientific notation with given significant figures."""
    if not np.isfinite(x) or x == 0.0:
        return r"0"
    sgn = "-" if x < 0 else ""
    ax = abs(x)
    exp = int(np.floor(np.log10(ax)))
    mant = ax / (10.0 ** exp)
    if exp == 0:
        return rf"{sgn}{mant:.{sig}f}"
    return rf"{sgn}{mant:.{sig}f}\times10^{{{exp}}}"

def compute_method_over_grid(
    x0_grid: np.ndarray,
    model: Tabulated1DModel,
    method_name: str,
    **extra_kwargs: Any,
) -> tuple[np.ndarray, np.ndarray]:
    """A wrapper to compute first derivatives over a grid of x0 values.

    This method uses DerivativeKit to compute the first derivative
    of the tabulated data at each x0 in x0_grid using the specified method
    of DerivativeKit.
    """
    slopes = np.empty_like(x0_grid, dtype=float)
    errs = np.full_like(x0_grid, np.nan, dtype=float)

    for i, x0 in enumerate(x0_grid):
        dk = DerivativeKit(function=model, x0=float(x0))
        try:
            val, err = dk.differentiate(
                method=method_name,
                order=1,
                return_error=True,
                **extra_kwargs,
            )
            slopes[i] = float(np.asarray(val).reshape(-1)[0])
            errs[i] = float(np.asarray(err).reshape(-1)[0])
        except TypeError:
            # method may not support return_error
            try:
                val = dk.differentiate(method=method_name, order=1, **extra_kwargs)
                slopes[i] = float(np.asarray(val).reshape(-1)[0])
                errs[i] = np.nan
            except Exception:
                slopes[i] = np.nan
                errs[i] = np.nan
        except Exception:
            slopes[i] = np.nan
            errs[i] = np.nan

    return slopes, errs


## 1) Build a noisy tabulated dataset

In [ ]:
apply_plot_style()  # apply DerivkKit plot style

rng = np.random.default_rng(42)  # random number generator for reproducibility, fixed seed at 42

n_tab = 70  # number of tabulated points
x_tab = np.linspace(0.0, 2.0 * np.pi, n_tab)  # x values on a regular grid
y_clean = sin_func(x_tab)  # clean sin(x) samples (perfect function values)

y_noise_sigma = 0.05  # a 5% noise level in y_tab
x_jitter_sigma = 0.0  # set >0 to enable x-jitter

y_noisy = y_clean + rng.normal(0.0, y_noise_sigma, size=y_clean.shape)

if x_jitter_sigma > 0:
    x_noisy = x_tab + rng.normal(0.0, x_jitter_sigma, size=x_tab.shape)
    srt = np.argsort(x_noisy)
    x_noisy = x_noisy[srt]
    y_noisy = y_noisy[srt]
else:
    x_noisy = x_tab.copy()

model_noisy = Tabulated1DModel(x_noisy, y_noisy, extrapolate=True)

x_noisy[:5], y_noisy[:5]


## 2) Evaluate derivatives across an interior grid of $x_0$ values

In [ ]:
n_eval = 50  # number of evaluation points for derivative estimates
# For the interior grid, avoid edges by 0.15 on each side
# just to be safe with finite-difference stencils
x0 = np.linspace(x_tab.min() + 0.15, x_tab.max() - 0.15, n_eval)
truth = df_truth(x0)

slopes_fr, errs_fr = compute_method_over_grid(
    x0, model_noisy, "finite", extrapolation="ridders"
)
slopes_ad, errs_ad = compute_method_over_grid(
    x0, model_noisy, "adaptive", n_points=27, spacing=0.25
)

m_fr = metrics(slopes_fr, truth)
m_ad = metrics(slopes_ad, truth)

print("Derivative accuracy vs truth:")
print(f"finite (Ridders): RMSE={m_fr['rmse']:.3e}  MAE={m_fr['mae']:.3e}  MaxAE={m_fr['maxae']:.3e}")
print(f"adaptive        : RMSE={m_ad['rmse']:.3e}  MAE={m_ad['mae']:.3e}  MaxAE={m_ad['maxae']:.3e}")


## 3) Plot 1 — noisy function table vs truth

In [ ]:
# colors (fallback to matplotlib defaults if style dict not present)
c_truth = DEFAULT_COLORS.get("red", None)
c_finite = DEFAULT_COLORS.get("yellow", None)

x_dense = np.linspace(x_tab.min(), x_tab.max(), 800)
y_dense = sin_func(x_dense)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.plot(x_dense, y_dense, lw=2.0, label=rf"truth $\sin(x)$", color=c_truth)
ax.plot(
    x_noisy,
    y_noisy,
    linestyle="--",
    linewidth=1.0,
    ms=5.0,
    label=rf"$y=\sin(x)+\mathcal{{N}}(0,{y_noise_sigma:g}^2)$",
    color=c_finite,
    marker="o",
    mfc="none",
    mec=c_finite,
    mew=1.2,
)

ax.set_xlabel("$x$")
ax.set_ylabel("$f(x)$")
ax.set_title("Exact function and noisy samples")
ax.legend(frameon=False, loc=1)
fig.tight_layout()
plt.show()


## 4) Plot 2 — derivative estimates (with internal errors when available) vs truth

In [ ]:
c_adaptive = DEFAULT_COLORS.get("blue", None)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.plot(x0, truth, lw=2.5, label=fr"truth $\cos(x)$", color=c_truth)

ax.errorbar(
    x0,
    slopes_fr,
    yerr=errs_fr,
    linestyle="--",
    linewidth=1.0,
    capsize=2,
    label="finite (Ridders)",
    color=c_finite,
    marker="o",
    mfc="none",
    mec=c_finite,
    mew=1.2,
    markersize=5,
)

ax.errorbar(
    x0,
    slopes_ad,
    yerr=errs_ad,
    linestyle="--",
    linewidth=1.0,
    capsize=2,
    label="adaptive",
    color=c_adaptive,
    marker="o",
    mfc="none",
    mec=c_adaptive,
    mew=1.2,
    markersize=5,
)

ax.set_xlabel("$x$")
ax.set_ylabel(r"$f'(x)$")
ax.set_title("Noisy tabulated derivative estimates vs truth")
ax.legend(frameon=False)
fig.tight_layout()
plt.show()


## 5) Plot 3 — method-reported internal error vs $x$

Legend includes RMSE and MAE (computed vs truth).

In [ ]:
lab_fr = (
    r"$\mathrm{finite\ (Ridders)}:$"
    "\n"
    + rf"$\mathrm{{RMSE}}={tex_sci(m_fr['rmse'])}$"
    "\n"
    + rf"$\mathrm{{MAE}}={tex_sci(m_fr['mae'])}$"
)

lab_ad = (
    r"$\mathrm{adaptive}:$"
    "\n"
    + rf"$\mathrm{{RMSE}}={tex_sci(m_ad['rmse'])}$"
    "\n"
    + rf"$\mathrm{{MAE}}={tex_sci(m_ad['mae'])}$"
)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.semilogy(
    x0,
    errs_fr,
    linestyle="--",
    linewidth=1.0,
    label=lab_fr,
    color=c_finite,
    marker="o",
    mfc="none",
    mec=c_finite,
    mew=1.2,
    markersize=5,
)

ax.semilogy(
    x0,
    errs_ad,
    linestyle="--",
    linewidth=1.0,
    label=lab_ad,
    color=c_adaptive,
    marker="o",
    mfc="none",
    mec=c_adaptive,
    mew=1.2,
    markersize=5,
)

ax.set_xlabel("$x$")
ax.set_ylabel(r"$\widehat{\sigma}_{f'}(x)$")
ax.set_title("Method-reported internal error vs $x$")
ax.legend(frameon=True, fontsize=11, loc=4)
fig.tight_layout()
plt.show()
